In [1]:
import json
import pandas as pd
import numpy as np
from IPython.display import IFrame, display
    
def float_formatter(x): return '{:.3f}'.format(x)

def highlight_max(s):
    if (s.dtype == object):
        is_max = [False for _ in range(s.shape[0])]
    else:
        is_max = s == s.max()
    return ['background: lightgreen' if cell else '' for cell in is_max]

wanted_keys = ("internal", "external")
dictfilt = lambda x, y: dict([ (i,'{:.2f}'.format(round(float(x[i]),2))) for i in x if i in set(y) ])

def retrieve_single_result_cnv():
    f = open(f'Result/Many_Feature_Selection_notebook/cnv/cnv.json')
    data = json.load(f)
    new_dict = {}
    new_dict['cnv'] = {}
    new_dict['cnv']['spearman'] = dictfilt(data['spearman'], wanted_keys)
    return pd.DataFrame(new_dict)

def print_best_estimator(dataset, feature_selection):
    f = open(f'Result/Many_Feature_Selection_notebook/{dataset}/{dataset}.json')
    data = json.load(f)
    print(data[feature_selection]['best_estimator'])
    
def retrieve_results():
    new_dict = {}
    for dataset in ['mirna', 'proteins', 'mrna', 'whole_dataset']:
        new_dict[dataset] = {}
        f = open(f'Result/Many_Feature_Selection_notebook/{dataset}/{dataset}.json')
        data = json.load(f)
        for metodo in ['mannwhitneyu','mannwhitneyu_with_umap', 'boruta', 'mine', 'mrmr']:
            if metodo in data:
                new_dict[dataset][metodo] = dictfilt(data[metodo], wanted_keys)
            else: 
                new_dict[dataset][metodo] = -1
    return pd.DataFrame(new_dict)

def load_filtered_normalize_data():
    dataset_dict = ['cnv','mirna', 'mrna', 'proteins']
    dict_test = {}
    for data in dataset_dict:
        with open('filtered_normalize_data/'+data+'.json', 'r') as fp:
            dataset = pd.read_json(fp)
            dict_test[data] = dataset
    return dict_test

def load_data_after_r_function():
    dataset_dict = ['cnv','mirna', 'mrna', 'proteins',
                    'dataset_whole_dataset', 'proteins_mirna_dataset',
                   'proteins_mirna_mrna_dataset']
    dict_test = {}
    for data in dataset_dict:
        with open('elaborated_data_with_r_function/r_'+data+'_removed.json', 'r') as fp:
            dataset = pd.read_json(fp)
            dict_test[data] = dataset
    return dict_test

lista_dataset = ['proteins', 'cnv', 'mrna', 'mirna','whole_dataset',
                'proteins_mirna','proteins_mirna_mrna']

lista_path_on_whole = ['Experiments_using_R_Function_intrinsic_and_umap_whole_dataset',
             'Experiments_using_R_Function_intrinsic_and_tsne_whole_dataset',
             'Experiments_using_R_Function_mannwhitney_and_multivariate_feature_selection_whole_dataset']

lista_path_on_fold = ['Experiments_using_R_Function_whole_intrinsic_and_umap_on_fold',
             'Experiments_using_R_Function_whole_intrinsic_and_tsne_on_fold',
             'Experiments_using_R_Function_mannwhitney_and_multivariate_feature_selection_on_fold']

def retrieve_results_on_fold_or_on_whole(lista_path, lista_dataset):
    new_dict = {}
    for dataset in lista_dataset:
        new_dict[dataset] = {}
        for path in lista_path:
            if 'umap' in path:
                f = open(f'Result/{path}/r_function_intrinsic_with_umap/{dataset}.json')
                data = json.load(f)
                new_dict[dataset]['r_function_intrinsic_with_umap'] = dictfilt(
                    data['r_function_intrinsic_with_umap'], wanted_keys)
            elif 'tsne' in path:
                f = open(f'Result/{path}/r_function_intrinsic_with_tsne/{dataset}.json')
                data = json.load(f)
                new_dict[dataset]['r_function_intrinsic_with_tsne'] = dictfilt(
                    data['r_function_intrinsic_with_tsne'], wanted_keys)
            elif 'multivariate' in path:
                if dataset != 'cnv':
                    
                    f = open(f'Result/{path}/r_function_with_mannwhitney_boruta/{dataset}.json')
                    data = json.load(f)
                    new_dict[dataset]['r_function_with_mannwhitney_boruta'] = dictfilt(
                        data['r_function_with_mannwhitney_boruta'], wanted_keys)
                    
                if dataset != 'mrna' and dataset != 'whole_dataset' and dataset != 'proteins_mirna_mrna':
                    
                    f = open(f'Result/{path}/r_function_with_mannwhitney_mrmr/{dataset}.json')
                    data = json.load(f)
                    new_dict[dataset]['r_function_with_mannwhitney_mrmr'] = dictfilt(
                        data['r_function_with_mannwhitney_mrmr'], wanted_keys)
                
                f = open(f'Result/{path}/r_function_with_mannwhitney/{dataset}.json')
                data = json.load(f)
                new_dict[dataset]['r_function_with_mannwhitney'] = dictfilt(
                    data['r_function_with_mannwhitney'], wanted_keys)
    return pd.DataFrame(new_dict)

datasets = load_filtered_normalize_data()

data_after_r_function = load_data_after_r_function()

#create combination of dataset
dataset_whole_dataset = pd.concat([datasets['cnv'], datasets['proteins'], datasets['mrna'], datasets['mirna']], axis=1)
datasets['dataset_whole_dataset'] = dataset_whole_dataset

proteins_mirna_dataset = pd.concat([datasets['proteins'], datasets['mirna']], axis=1)
datasets['proteins_mirna_dataset'] = proteins_mirna_dataset

proteins_mirna_mrna_dataset = pd.concat([datasets['proteins'], datasets['mirna'], datasets['mrna']], axis=1)
datasets['proteins_mirna_mrna_dataset'] = proteins_mirna_mrna_dataset

# Osservazioni iniziali

1) Inizialmente tutti i i dati sono stati normalizzati usando MinMaxScaler e filtrati scartando le feature con bassa variabilità ( deviazione standard < 0.05)
3) Il modello usato è sempre RandomForest
3) Viene usata l'AUPRC come misuratore di performance. La baseline per il dataset fornito è 0.122807, ovvero il 12% dei pazienti sono positivi (77 su 627).
4) Viene sempre applicata una StratifiedKFold usando 10 fold.
5) I risultati sono sempre presentati con etichetta "external" e "internal". Questo sta a indicare quando è stata calcolata la AUPRC: nel primo caso le predizioni effettuate dal modello nei 10 fold vengono concatenate e una volta finite viene calcolata l'AUPRC totale; nel secondo caso invece ad ogni iterazione viene calcolata l'AUPRC e alla fine viene calcolata la media. 

### A cnv ho applicato spearman e scartato le feature altamente correlate tra di loro o bassamente correlate con la label

In [2]:
retrieve_single_result_cnv()

,cnv
spearman,"{'internal': '0.13', 'external': '0.10'}"


In [3]:
print_best_estimator('cnv', 'spearman')

RandomForestClassifier(criterion='entropy', max_leaf_nodes=5, n_estimators=101)


### Figura

In [4]:
filepath = "Result/Many_Feature_Selection_notebook/cnv/spearman_AUPRC_External.pdf"
IFrame(filepath, width=700, height=400)

# Risultati di proteins, miRNA e mRNA e concat (concatenazione di tutti i dataset)

In [5]:
retrieve_results()

,mirna,proteins,mrna,whole_dataset
mannwhitneyu,"{'internal': '0.25', 'external': '0.16'}","{'internal': '0.22', 'external': '0.15'}","{'internal': '0.23', 'external': '0.15'}","{'internal': '0.23', 'external': '0.16'}"
mannwhitneyu_with_umap,"{'internal': '0.27', 'external': '0.12'}","{'internal': '0.26', 'external': '0.12'}","{'internal': '0.39', 'external': '0.14'}","{'internal': '0.21', 'external': '0.11'}"
boruta,"{'internal': '0.18', 'external': '0.14'}","{'internal': '0.17', 'external': '0.12'}","{'internal': '0.20', 'external': '0.16'}","{'internal': '0.22', 'external': '0.16'}"
mine,"{'internal': '0.25', 'external': '0.17'}","{'internal': '0.21', 'external': '0.15'}","{'internal': '0.23', 'external': '0.17'}",-1
mrmr,"{'internal': '0.20', 'external': '0.14'}","{'internal': '0.19', 'external': '0.14'}",-1,-1


#### Per mRNA e concat in corrispodenza di mrmr c'è -1, questo perchè il calcolo richiedeva tantissimo tempo, sopratutto usando 10 fold

## Best estimator of the best combination of feature selection + extraction

In [6]:
print_best_estimator('proteins', 'mannwhitneyu_with_umap')

RandomForestClassifier(criterion='entropy', max_features='log2',
                       max_leaf_nodes=5, n_estimators=51)


In [7]:
print_best_estimator('mrna', 'mannwhitneyu_with_umap')

RandomForestClassifier(max_leaf_nodes=10, n_estimators=51)


In [8]:
print_best_estimator('mirna', 'mannwhitneyu_with_umap')

RandomForestClassifier(criterion='entropy', max_features='log2',
                       max_leaf_nodes=10, n_estimators=51)


In [9]:
print_best_estimator('whole_dataset', 'mannwhitneyu_with_umap')

RandomForestClassifier(max_leaf_nodes=10, n_estimators=51)


# Seconda tipologia di esperimenti

### Come prima cosa viene utlizzata una funzione che scarta le feature altamente correlate tra di loro usando "pearson" con un cutoff = 0.8. Vediamo quante feature vengono scartate:

In [10]:
for dataset in ['proteins', 'cnv', 'mirna', 'mrna',
                'dataset_whole_dataset', 'proteins_mirna_dataset',
               'proteins_mirna_mrna_dataset']:    
    print(f"{dataset}: feature prima dell applicazione della funzione: {len(datasets[dataset].columns)}")
    print(f"{dataset}: feature dopo l applicazione della funzione: {len(data_after_r_function[dataset].columns)}")
    print(f"{dataset}: feature eliminate: {len(datasets[dataset].columns)-len(data_after_r_function[dataset].columns)}")
    print('\n')



proteins: feature prima dell applicazione della funzione: 216
proteins: feature dopo l applicazione della funzione: 216
proteins: feature eliminate: 0


cnv: feature prima dell applicazione della funzione: 24776
cnv: feature dopo l applicazione della funzione: 194
cnv: feature eliminate: 24582


mirna: feature prima dell applicazione della funzione: 773
mirna: feature dopo l applicazione della funzione: 761
mirna: feature eliminate: 12


mrna: feature prima dell applicazione della funzione: 18465
mrna: feature dopo l applicazione della funzione: 18411
mrna: feature eliminate: 54


dataset_whole_dataset: feature prima dell applicazione della funzione: 44230
dataset_whole_dataset: feature dopo l applicazione della funzione: 20386
dataset_whole_dataset: feature eliminate: 23844


proteins_mirna_dataset: feature prima dell applicazione della funzione: 989
proteins_mirna_dataset: feature dopo l applicazione della funzione: 977
proteins_mirna_dataset: feature eliminate: 12


proteins_mirna_m

### La differenza tra on fold e on whole dataset:
1) In entrambi i casi la funzione che calcola la correlazione tra le feature viene applicata sull'intero dataset, prima di darlo in pasto alla funzione che splitta i dati in fold e addestra
2) In 'on fold' il calcolo della dimensione intrinseca viene effettuato sul train set del fold generato durante la StratifiedKFold e la dimensione calcolata viene usata per fare feature extraction sia sul train set che sul test set, usando tSNE e Umap.
3) In 'on whole dataset' invece la dimensione intrinseca viene calcolata esternamente alla funzione, quindi su tutto il dataset, e anche la feature extraction viene effettuata esternamente. Il dataset cosi ottenuto viene passato alla funzione che addestra.

# On fold

In [11]:
retrieve_results_on_fold_or_on_whole(lista_path_on_fold, lista_dataset)

,proteins,cnv,mrna,mirna,whole_dataset,proteins_mirna,proteins_mirna_mrna
r_function_intrinsic_with_umap,"{'internal': '0.35', 'external': '0.14'}","{'internal': '0.20', 'external': '0.12'}","{'internal': '0.32', 'external': '0.21'}","{'internal': '0.38', 'external': '0.17'}","{'internal': '0.23', 'external': '0.17'}","{'internal': '0.37', 'external': '0.12'}","{'internal': '0.36', 'external': '0.22'}"
r_function_intrinsic_with_tsne,"{'internal': '0.20', 'external': '0.14'}","{'internal': '0.16', 'external': '0.11'}","{'internal': '0.18', 'external': '0.13'}","{'internal': '0.16', 'external': '0.13'}","{'internal': '0.16', 'external': '0.14'}","{'internal': '0.18', 'external': '0.12'}","{'internal': '0.16', 'external': '0.12'}"
r_function_with_mannwhitney_boruta,"{'internal': '0.22', 'external': '0.15'}",NaN,"{'internal': '0.22', 'external': '0.18'}","{'internal': '0.17', 'external': '0.13'}","{'internal': '0.22', 'external': '0.15'}","{'internal': '0.16', 'external': '0.13'}","{'internal': '0.20', 'external': '0.15'}"
r_function_with_mannwhitney_mrmr,"{'internal': '0.21', 'external': '0.16'}","{'internal': '0.16', 'external': '0.11'}",NaN,"{'internal': '0.25', 'external': '0.17'}",NaN,"{'internal': '0.22', 'external': '0.16'}",NaN
r_function_with_mannwhitney,"{'internal': '0.21', 'external': '0.15'}","{'internal': '0.15', 'external': '0.12'}","{'internal': '0.21', 'external': '0.16'}","{'internal': '0.23', 'external': '0.17'}","{'internal': '0.22', 'external': '0.17'}","{'internal': '0.25', 'external': '0.17'}","{'internal': '0.24', 'external': '0.18'}"


### whole dataset 

In [12]:
retrieve_results_on_fold_or_on_whole(lista_path_on_whole, lista_dataset)

,proteins,cnv,mrna,mirna,whole_dataset,proteins_mirna,proteins_mirna_mrna
r_function_intrinsic_with_umap,"{'internal': '0.72', 'external': '0.70'}","{'internal': '0.55', 'external': '0.56'}","{'internal': '0.63', 'external': '0.64'}","{'internal': '0.79', 'external': '0.79'}","{'internal': '0.70', 'external': '0.68'}","{'internal': '0.73', 'external': '0.73'}","{'internal': '0.70', 'external': '0.68'}"
r_function_intrinsic_with_tsne,"{'internal': '0.14', 'external': '0.11'}","{'internal': '0.15', 'external': '0.12'}","{'internal': '0.17', 'external': '0.11'}","{'internal': '0.21', 'external': '0.15'}","{'internal': '0.16', 'external': '0.14'}","{'internal': '0.19', 'external': '0.14'}","{'internal': '0.18', 'external': '0.14'}"
r_function_with_mannwhitney_boruta,"{'internal': '0.24', 'external': '0.17'}",NaN,"{'internal': '0.38', 'external': '0.29'}","{'internal': '0.29', 'external': '0.18'}","{'internal': '0.39', 'external': '0.30'}","{'internal': '0.30', 'external': '0.24'}","{'internal': '0.38', 'external': '0.32'}"
r_function_with_mannwhitney_mrmr,"{'internal': '0.23', 'external': '0.17'}","{'internal': '0.26', 'external': '0.16'}",NaN,"{'internal': '0.25', 'external': '0.18'}",NaN,"{'internal': '0.29', 'external': '0.22'}",NaN
r_function_with_mannwhitney,"{'internal': '0.25', 'external': '0.19'}","{'internal': '0.24', 'external': '0.17'}","{'internal': '0.26', 'external': '0.18'}","{'internal': '0.22', 'external': '0.18'}","{'internal': '0.26', 'external': '0.16'}","{'internal': '0.31', 'external': '0.21'}","{'internal': '0.25', 'external': '0.17'}"
